## Derive Experiment Dataset from Landy et al. (2022) Cryosat-2 product

This notebook is designed to explore the Landy et al. (2022) Cryosat-2 filled-in summer gridded SIT record.

The goals are to:
1) Calculate area, volume, total SIA, total SIV, and thickness and generate a netcdf dataset for analysis.
2) Standardize the time field 
3) Make some preliminary plots that outline the data

### Prepare Notebook

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import glob 
import datetime

### Convert the Landy et al. (2022) dataset

Read in the Landy et al. (2022) original dataset

In [2]:
file = '/glade/scratch/mollyw/external_data/SIT/UBristolCS2/ubristol_cryosat2_seaicethickness_nh_80km_v1p7.nc'
orig_ds = xr.open_dataset(file, decode_times=False)

Look at the data.

In [3]:
orig_ds

<xarray.Dataset>
Dimensions:                        (t: 236, y: 100, x: 100)
Dimensions without coordinates: t, y, x
Data variables:
    Time                           (t) float64 ...
    Mode                           (t) float64 ...
    Latitude                       (y, x) float64 ...
    Longitude                      (y, x) float64 ...
    Sea_Ice_Thickness              (t, y, x) float64 ...
    Sea_Ice_Thickness_Uncertainty  (t, y, x) float64 ...
    Sea_Ice_Concentration          (t, y, x) float64 ...
    Sea_Ice_Type                   (t, y, x) float64 ...
Attributes: (12/36)
    Dataset:                   Sea ice thickness observations derived from Cr...
    Date_Created:              18-Aug-2021
    Source_Altimetry:          Cryosat-2 SIRAL Baseline-D L1B SAR and SARIn W...
    Source_Snow:               Distributed Snow Evolution Model (SnowModel-LG...
    Source_Ice_Concentration:  OSI-SAF Global Sea Ice Concentration (OSI-401-b)
    Source_Ice_Type:           NSIDC Sea Ice Age v4.1
    ...                        ...
    publisher_name:            NERC EDS UK Polar Data Centre
    publisher_type:            institution
    publisher_email:           pdc@bas.ac.uk
    publisher_url:             https://www.bas.ac.uk/data/uk-pdc/
    references:                Please cite the following publications if usin...
    citation:                  Landy, J., & Dawson, G. (2022). Year-round Arc...

Fix the time dimension. Convert to datetime.

In [4]:
# set an inital datetime object
date_init = datetime.datetime(1,1,1)

# generate an array of datetime objects using the "days since Jan 0, year 0" baseline
cs2_dates = np.array([date_init+ datetime.timedelta(days=(x-366)) for x in list(orig_ds.Time.values)])

Replace time in the original data set and add in the area variable.

In [5]:
adj_ds = orig_ds.copy()
adj_ds = adj_ds.rename({'t':'time'})
adj_ds['time'] = cs2_dates
adj_ds = adj_ds.drop('Time')

Look at the adjusted dataset.

In [6]:
adj_ds

<xarray.Dataset>
Dimensions:                        (time: 236, y: 100, x: 100)
Coordinates:
  * time                           (time) datetime64[ns] 2010-10-09 ... 2020-...
Dimensions without coordinates: y, x
Data variables:
    Mode                           (time) float64 1.0 1.0 1.0 ... 2.0 2.0 2.0
    Latitude                       (y, x) float64 41.34 41.77 ... 41.77 41.34
    Longitude                      (y, x) float64 -45.0 -45.58 ... 134.4 135.0
    Sea_Ice_Thickness              (time, y, x) float64 ...
    Sea_Ice_Thickness_Uncertainty  (time, y, x) float64 ...
    Sea_Ice_Concentration          (time, y, x) float64 ...
    Sea_Ice_Type                   (time, y, x) float64 ...
Attributes: (12/36)
    Dataset:                   Sea ice thickness observations derived from Cr...
    Date_Created:              18-Aug-2021
    Source_Altimetry:          Cryosat-2 SIRAL Baseline-D L1B SAR and SARIn W...
    Source_Snow:               Distributed Snow Evolution Model (SnowModel-LG...
    Source_Ice_Concentration:  OSI-SAF Global Sea Ice Concentration (OSI-401-b)
    Source_Ice_Type:           NSIDC Sea Ice Age v4.1
    ...                        ...
    publisher_name:            NERC EDS UK Polar Data Centre
    publisher_type:            institution
    publisher_email:           pdc@bas.ac.uk
    publisher_url:             https://www.bas.ac.uk/data/uk-pdc/
    references:                Please cite the following publications if usin...
    citation:                  Landy, J., & Dawson, G. (2022). Year-round Arc...

### Calculate desired variables

Calculate gridded sea ice area and volume, and corresponding pan-Arctic totals.

In [7]:
def lat_lon_cell_area(lat_lon_grid_cell):
    """
    Calculate the area of a cell, in meters^2, on a lat/lon grid.
    
    This applies the following equation from Santinie et al. 2010.
    
    S = (λ_2 - λ_1)(sinφ_2 - sinφ_1)R^2
    
    S = surface area of cell on sphere
    λ_1, λ_2, = bands of longitude in radians
    φ_1, φ_2 = bands of latitude in radians
    R = radius of the sphere
    
    Santini, M., Taramelli, A., & Sorichetta, A. (2010). ASPHAA: A GIS‐Based 
    Algorithm to Calculate Cell Area on a Latitude‐Longitude (Geographic) 
    Regular Grid. Transactions in GIS, 14(3), 351-377.
    https://doi.org/10.1111/j.1467-9671.2010.01200.x

    Parameters
    ----------
    lat_lon_grid_cell
        A shapely box with coordinates on the lat/lon grid

    Returns
    -------
    float
        The cell area in meters^2

    """
    from numpy import radians, cos, sin
    
    # mean earth radius - https://en.wikipedia.org/wiki/Earth_radius#Mean_radius
    AVG_EARTH_RADIUS_METERS = 6371008.8
    
    west, south, east, north = lat_lon_grid_cell.bounds
    
    if abs(west - east) > 300:
        west = west + 360
        # print(east)
    
    west = radians(west)
    east = radians(east)
    south = radians(south)
    north = radians(north)
    
    return (east - west) * (sin(north) - sin(south)) * (AVG_EARTH_RADIUS_METERS**2)

In [8]:
from shapely.geometry import box as BOX

In [9]:
grid_cell_area = np.zeros([100,100])
for x in range(0, 100-1):
    for y in range(0,100-1):
        lat1 = adj_ds.Latitude[x,y]
        lat2 = adj_ds.Latitude[x+1,y+1]
     
        
        lon1 = adj_ds.Longitude[x,y]
        lon2 = adj_ds.Longitude[x+1,y+1]
        
        box = BOX(lon1, lat1, lon2, lat2)
        
        grid_cell_area[x,y] = lat_lon_cell_area(box)

In [10]:
# for now, choose a standard grid cell area size of 80 km^2
adj_ds['grid_area'] = (('x','y'),grid_cell_area)

# Calculate gridded and total area and volume
Sea_Ice_Area = adj_ds.Sea_Ice_Concentration * grid_cell_area
Sea_Ice_Volume = Sea_Ice_Area * adj_ds.Sea_Ice_Thickness
Total_SIA = Sea_Ice_Area.sum(dim = ['x','y'])
Total_SIV = Sea_Ice_Volume.sum(dim = ['x','y'])

In [11]:
adj_ds['siarea'] = Sea_Ice_Area
adj_ds['sivol'] = Sea_Ice_Volume
adj_ds['SIA'] = Total_SIA
adj_ds['SIV'] = Total_SIV

In [12]:
adj_ds = adj_ds.rename({'Sea_Ice_Thickness':'sithick', 
                        'Sea_Ice_Concentration':'siconc', 
                        'Sea_Ice_Type':'ice_type',
                        'Sea_Ice_Thickness_Uncertainty':'sit_unc',
                        'Latitude':'latitude', 
                        'Longitude':'longitude',
                        'Mode':'algorithm'})
adj_ds = adj_ds.set_coords(('latitude','longitude'))
adj_ds = adj_ds.transpose('time','x','y')

In [13]:
adj_ds

<xarray.Dataset>
Dimensions:    (time: 236, y: 100, x: 100)
Coordinates:
    latitude   (x, y) float64 41.34 41.77 42.2 42.63 ... 42.63 42.2 41.77 41.34
    longitude  (x, y) float64 -45.0 -44.42 -43.82 -43.21 ... 136.2 135.6 135.0
  * time       (time) datetime64[ns] 2010-10-09 2010-10-24 ... 2020-07-24
Dimensions without coordinates: y, x
Data variables:
    algorithm  (time) float64 1.0 1.0 1.0 1.0 1.0 1.0 ... 2.0 2.0 2.0 2.0 2.0
    sithick    (time, x, y) float64 ...
    sit_unc    (time, x, y) float64 ...
    siconc     (time, x, y) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    ice_type   (time, x, y) float64 ...
    grid_area  (x, y) float64 0.0 9.794e+07 1.992e+08 3.037e+08 ... 0.0 0.0 0.0
    siarea     (time, x, y) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    sivol      (time, x, y) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    SIA        (time) float64 3.434e+12 4.219e+12 ... 4.413e+12 3.336e+12
    SIV        (time) float64 1.103e+12 2.838e+12 ... 2.919e+12 1.688e+12
Attributes: (12/36)
    Dataset:                   Sea ice thickness observations derived from Cr...
    Date_Created:              18-Aug-2021
    Source_Altimetry:          Cryosat-2 SIRAL Baseline-D L1B SAR and SARIn W...
    Source_Snow:               Distributed Snow Evolution Model (SnowModel-LG...
    Source_Ice_Concentration:  OSI-SAF Global Sea Ice Concentration (OSI-401-b)
    Source_Ice_Type:           NSIDC Sea Ice Age v4.1
    ...                        ...
    publisher_name:            NERC EDS UK Polar Data Centre
    publisher_type:            institution
    publisher_email:           pdc@bas.ac.uk
    publisher_url:             https://www.bas.ac.uk/data/uk-pdc/
    references:                Please cite the following publications if usin...
    citation:                  Landy, J., & Dawson, G. (2022). Year-round Arc...

Save the data to a new, local netcdf file.

In [14]:
adj_ds.to_netcdf('cs2_landy_data.nc')